# Function Testing Notebook - Yong copy

This notebook is meant to test external-facing functions to ensure they are working as expected.

A dedicated test_all_functions.py with unit testing might be better, but this is a good benchmark

In [1]:
%matplotlib qt

In [2]:
import os
import sys
from pathlib import Path
import time
import tempfile
import logging
import glob

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mne
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, LocalCluster
import joblib

In [3]:
# packageroot = Path('../../').resolve()
packageroot = Path('/Users/ohy2/Documents/GitHubRepo/PyEEG').resolve()
print(packageroot)
sys.path.append(str(packageroot))

from pythoneeg import core
from pythoneeg import visualization
# from pythoneeg import constants

print(core.__all__)
# print(visualization.__all__)
# print(dir(constants))

core.set_temp_directory('/Users/ohy2/Documents/GitHubRepo/PyEEG')

/Users/ohy2/Documents/GitHubRepo/PyEEG


/Users/ohy2/Documents/GitHubRepo/PyEEG/.venv_mouseEEG/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['DDFBinaryMetadata', 'LongRecordingOrganizer', 'convert_ddfcolbin_to_ddfrowbin', 'convert_ddfrowbin_to_si', 'convert_units_to_multiplier', 'convert_colpath_to_rowpath', 'filepath_to_index', 'is_day', 'set_temp_directory', 'get_temp_directory', 'LongRecordingAnalyzer', 'MountainSortAnalyzer', 'FragmentAnalyzer']


In [ ]:
# %%script true
# animal_ids = ['A5', 'A10', 'F22', 'G25']
# animal_ids = ['A5']
animal_ids = ['A5', 'A10']
# animal_ids = ['F22']
# animal_ids = ['G25']
base_folder = Path('./test-data').resolve()
# base_folder = Path('/mnt/isilon/marsh_single_unit/PythonEEG Data Bins').resolve()
for animal_id in animal_ids:
    ao = visualization.AnimalOrganizer(base_folder, animal_id, mode="concat", assume_from_number=True, truncate=False)
    ao.convert_colbins_to_rowbins(overwrite=False)
    ao.convert_rowbins_to_rec() # paralleization breaks if not enough memory

    # with Client(cluster_window) as client:
    #     client.upload_file(str(packageroot / 'pythoneeg.zip'))
    #     display(client)
    #     print("running war")
    #     war = ao.compute_windowed_analysis(['all'], exclude=['nspike', 'wavetemp'], multiprocess_mode='dask')
    #     war.to_pickle_and_json(Path(f'./test-wars/{animal_id}').resolve())

    # with Client(cluster_spike) as client:
    #     client.upload_file(str(packageroot / 'pythoneeg.zip'))
    #     display(client)
    #     print("running sar")
    #     sar = ao.compute_spike_analysis(multiprocess_mode='dask')

    # with Client(LocalCluster()) as client:
    #     client.upload_file(str(packageroot / 'pythoneeg.zip'))
    #     display(client)
    #     print("running sar")
    sar = ao.compute_spike_analysis(multiprocess_mode='serial')

In [ ]:
for e in sar:
    e.convert_to_mne(chunk_len=1440)
    e.save_fif_and_json(Path(f'./test-mnes/{e.animal_day}').resolve())

In [ ]:
reconstruct_sas = visualization.SpikeAnalysisResult.load_fif_and_json('./test-mnes/A5 WT Dec-12-2023')
mne_obj = reconstruct_sas.result_mne



print(mne_obj.info)

# Apply the custom function to your MNE object
#mne_obj.rename_channels(core.utils.clean_channel_name)


In [ ]:
# Create a list to store all mne objects
mne_objects = []

animal_ids = ['A5', 'A10']

# Get all files for each animal ID
for animal_id in animal_ids:  # ['A5', 'A10']
    # Find all files matching the pattern for this animal
    file_pattern = f'./test-mnes/{animal_id}*'
    animal_files = glob.glob(file_pattern)
    
    print(f"Found {len(animal_files)} files for {animal_id}:")
    for file in animal_files:
        print(f"  - {file}")
        
        # Load each file and add to our list
        try:
            reconstruct_sas = visualization.SpikeAnalysisResult.load_fif_and_json(file)
            mne_obj = reconstruct_sas.result_mne
            
            # Store metadata in the 'temp' dictionary
            if 'temp' not in mne_obj.info:
                mne_obj.info['temp'] = {}
            mne_obj.info['temp']['animal_id'] = animal_id
            mne_obj.info['temp']['file_path'] = file
            
            # Add to our list
            mne_objects.append(mne_obj)
            print(f"  ✓ Successfully loaded")
        except Exception as e:
            print(f"  ✗ Error loading: {str(e)}")

print(f"\nTotal MNE objects loaded: {len(mne_objects)}")


In [ ]:
# Now you can access the metadata like this:
for i, obj in enumerate(mne_objects):
    print(f"Object {i}: Animal {obj.info['temp']['animal_id']}, from {obj.info['temp']['file_path']}")

In [ ]:
events, event_id = mne.events_from_annotations(raw=mne_obj)
#fig = mne.viz.plot_events(events, sfreq=mne_obj.info['sfreq'], first_samp=mne_obj.first_samp, event_id=event_id)

unique_events, counts = np.unique(events[:, 2], return_counts=True)

print("Event counts:")
for event_code, count in zip(unique_events, counts):
    for name, code in event_id.items():
        if code == event_code:
            print(f"{name}: {count}")

In [4]:
def random_subsample_epochs(epochs: mne.Epochs, max_epochs=1000):
    if len(epochs) > max_epochs:
        # Get random indices for subsampling
        indices = np.random.choice(len(epochs), size=max_epochs, replace=False)
        epochs = epochs[indices]
    return epochs

In [13]:
def reject_epochs_by_rms(epochs, threshold_sd=2.5, return_scores=True):
    """
    Reject epochs based on RMS amplitude exceeding a threshold in standard deviations,
    computing thresholds separately for each channel.
    
    Parameters:
    -----------
    epochs : mne.Epochs
        The epochs object to clean
    threshold_sd : float
        Threshold in standard deviations from mean RMS
    return_scores : bool
        Whether to return the RMS scores for all epochs and channels
        
    Returns:
    --------
    epochs_clean : mne.Epochs
        The cleaned epochs object
    rejection_info : dict
        Dictionary with rejection statistics
    rms_scores : ndarray (optional)
        RMS amplitude for each epoch and channel if return_scores=True
    """
    import numpy as np
    
    # Make sure data is loaded
    epochs.load_data()
    
    # Get the data
    data = epochs.get_data()
    n_epochs, n_channels, n_times = data.shape
    
    # Compute RMS for each epoch and each channel
    # Formula: RMS = sqrt(mean(x²))
    rms_per_epoch_channel = np.sqrt(np.mean(np.square(data), axis=2))  # shape: (n_epochs, n_channels)
    
    # Compute mean and standard deviation of RMS values for each channel
    rms_mean_per_channel = np.mean(rms_per_epoch_channel, axis=0)
    rms_std_per_channel = np.std(rms_per_epoch_channel, axis=0)
    
    # Set upper and lower thresholds for each channel
    upper_thresh_per_channel = rms_mean_per_channel + threshold_sd * rms_std_per_channel
    lower_thresh_per_channel = rms_mean_per_channel - threshold_sd * rms_std_per_channel
    
    # Find bad epochs per channel
    bad_epochs_mask_per_channel = (rms_per_epoch_channel > upper_thresh_per_channel) | \
                                 (rms_per_epoch_channel < lower_thresh_per_channel)
    
    # An epoch is considered bad if it's bad in any channel
    bad_epochs_mask = np.any(bad_epochs_mask_per_channel, axis=1)
    
    # Get indices of epochs to drop
    bad_indices = np.where(bad_epochs_mask)[0]
    
    # Create a clean copy with the bad epochs dropped
    epochs_clean = epochs.copy().drop(bad_indices)
    
    # Count bad epochs per channel
    bad_epochs_per_channel = np.sum(bad_epochs_mask_per_channel, axis=0)
    percent_bad_per_channel = (bad_epochs_per_channel / n_epochs) * 100
    
    # Overall rejection statistics
    n_epochs_rejected = len(bad_indices)
    percent_rejected = (n_epochs_rejected / n_epochs) * 100
    
    # Create info dictionary
    channel_info = {
        ch_name: {
            'bad_epochs': bad_epochs_per_channel[i],
            'percent_bad': percent_bad_per_channel[i],
            'rms_mean': rms_mean_per_channel[i],
            'rms_std': rms_std_per_channel[i],
            'upper_threshold': upper_thresh_per_channel[i],
            'lower_threshold': lower_thresh_per_channel[i]
        }
        for i, ch_name in enumerate(epochs.ch_names)
    }
    
    rejection_info = {
        'n_epochs_original': n_epochs,
        'n_epochs_rejected': n_epochs_rejected,
        'n_epochs_kept': n_epochs - n_epochs_rejected,
        'percent_rejected': percent_rejected,
        'threshold_sd': threshold_sd,
        'bad_indices': bad_indices,
        'channels': channel_info
    }
    
    # Print rejection statistics
    print(f"Epochs rejection summary:")
    print(f"  Original epochs: {n_epochs}")
    print(f"  Rejected epochs: {n_epochs_rejected} ({percent_rejected:.2f}%)")
    print(f"  Remaining epochs: {n_epochs - n_epochs_rejected}")
    print(f"  Rejection by channel:")
    
    # Sort channels by percent bad
    sorted_channels = sorted(
        epochs.ch_names, 
        key=lambda ch: channel_info[ch]['percent_bad'],
        reverse=True
    )
    
    for ch_name in sorted_channels:
        info = channel_info[ch_name]
        print(f"    {ch_name}: {info['bad_epochs']} epochs ({info['percent_bad']:.2f}%)")
    
    if return_scores:
        return epochs_clean, rejection_info, rms_per_epoch_channel
    else:
        return epochs_clean, rejection_info


def epoch_peri_spike(mne_obj: mne.io.Raw, tmin=-10, tmax=5, baseline=(None, -5), l_freq=1, h_freq=50):

    raw = mne_obj.copy()
    
    # with joblib.parallel_backend('dask', scheduler_host=cluster_general.scheduler_address):
    # raw.filter(l_freq=l_freq, h_freq=h_freq)

    # logging.debug(f"Events: {events}")
    # logging.debug(f"Event dict: {event_dict}")

    # Process each spike type separately
        # logging.debug(f"Processing channel: {channel_name}")
        
    # Create spike-centered epochs
    epochs = mne.Epochs(
        raw,
        tmin=tmin,
        tmax=tmax,
        baseline=baseline,
        event_repeated='drop',
        preload=False)
    epochs.drop_bad()
    #epochs = random_subsample_epochs(epochs).load_data()
    
    # with joblib.parallel_backend('dask', scheduler_host=cluster_general.scheduler_address):
    epochs.filter(l_freq=l_freq, h_freq=h_freq)
    
    return epochs

def plot_peri_spike_evoked(epochs: mne.epochs, tcrop=(-3, 3)):
    
    # Create a figure with subplots for each channel
    n_channels = len(epochs.ch_names)
    fig, axes = plt.subplots(n_channels, 1, figsize=(10, 2*n_channels), sharex=True)

    for ch_idx, ch_name in enumerate(epochs.ch_names):
        evoked = epochs[ch_name].average()
        evoked.crop(tmin=tcrop[0], tmax=tcrop[1])
        evoked.plot(picks=[ch_idx], axes=axes[ch_idx], show=False, spatial_colors=False)
        axes[ch_idx].set_title(ch_name)

    plt.tight_layout()
    plt.show()


def plot_peri_spike_tfr(epochs, l_freq=1, h_freq=50, baseline=(-10, -5), tcrop=(-3, 3), 
                        rescale_method='zscore', fmin=None, fmax=None):
    """
    Plot time-frequency representation around spikes with multiple rescaling options.
    
    Parameters:
    -----------
    epochs : mne.Epochs
        The epoch data around spikes
    l_freq : float
        Lower frequency bound
    h_freq : float
        Upper frequency bound
    baseline : tuple
        Time range to use for baseline (for normalization)
    tcrop : tuple
        Time range to display in plots
    rescale_method : str
        Method for rescaling data:
        - 'log': Apply log10 transform
        - 'db': Convert to decibels relative to baseline (10*log10(data/baseline))
        - 'zscore': Z-score relative to baseline (default)
        - 'percent': Percent change from baseline
        - 'ratio': Ratio relative to baseline (data/baseline)
        - 'none': No rescaling
    fmin : float | None
        Minimum frequency to display (if None, use l_freq)
    fmax : float | None
        Maximum frequency to display (if None, use h_freq)
    
    Returns:
    --------
    fig : matplotlib.Figure
        The figure containing the plots
    """
    # Set frequency range for display
    fmin = l_freq if fmin is None else fmin
    fmax = h_freq if fmax is None else fmax
    
    # Create a figure with subplots for each channel
    n_channels = len(epochs.ch_names)
    fig, axes = plt.subplots(n_channels, 1, figsize=(10, 2*n_channels), sharex=True)
    
    # Ensure axes is always a list for consistent indexing
    if n_channels == 1:
        axes = [axes]
    
    # Frequencies to analyze
    freqs = np.arange(l_freq, h_freq, 1)
    
    # Set colormap based on rescale method
    if rescale_method in ['zscore', 'percent']:
        cmap = 'RdBu_r'  # Diverging colormap for data centered around zero
    else:
        cmap = 'viridis'  # Sequential colormap for non-centered data
    
    for ch_idx, ch_name in enumerate(epochs.ch_names):
        # Compute TFR using Morlet wavelets
        tfr_spike = epochs[ch_name].compute_tfr(
            freqs=freqs, 
            n_cycles=freqs / 2,  # Adjust n_cycles based on frequency 
            method='morlet', 
            use_fft=True, 
            average=True
        )
        
        # Get the data and apply requested transform
        data = tfr_spike.data.copy()
        
        # Handle small/zero values for log transforms
        if rescale_method in ['log', 'db', 'ratio', 'percent']:
            # Add small constant to avoid log(0)
            epsilon = np.finfo(float).eps
            data = np.maximum(data, epsilon)
        
        # Apply selected rescaling method
        if rescale_method == 'log':
            # Simple log10 transform
            data = np.log10(data)
            colorbar_label = 'log₁₀(Power)'
        
        elif rescale_method == 'db':
            # Convert to decibels relative to baseline
            # First compute baseline power
            baseline_mask = ((tfr_spike.times >= baseline[0]) & 
                            (tfr_spike.times <= baseline[1]))
            baseline_data = np.mean(data[:, :, baseline_mask], axis=2, keepdims=True)
            # Then compute dB = 10*log10(data/baseline)
            data = 10 * np.log10(data / baseline_data)
            colorbar_label = 'Power (dB)'
            # Use diverging colormap for dB
            cmap = 'RdBu_r'
        
        elif rescale_method == 'zscore':
            # Z-score relative to baseline using MNE's function
            data = mne.baseline.rescale(
                data,
                tfr_spike.times,
                baseline=baseline,
                mode='zscore'
            )
            colorbar_label = 'Z-score'
        
        elif rescale_method == 'percent':
            # Percent change from baseline
            baseline_mask = ((tfr_spike.times >= baseline[0]) & 
                            (tfr_spike.times <= baseline[1]))
            baseline_data = np.mean(data[:, :, baseline_mask], axis=2, keepdims=True)
            data = 100 * (data - baseline_data) / baseline_data
            colorbar_label = 'Change (%)'
        
        elif rescale_method == 'ratio':
            # Simple ratio to baseline
            baseline_mask = ((tfr_spike.times >= baseline[0]) & 
                            (tfr_spike.times <= baseline[1]))
            baseline_data = np.mean(data[:, :, baseline_mask], axis=2, keepdims=True)
            data = data / baseline_data
            colorbar_label = 'Ratio to baseline'
        
        elif rescale_method == 'none':
            # No rescaling
            colorbar_label = 'Power'
        
        else:
            raise ValueError(f"Unknown rescale method: {rescale_method}")
        
        # Now crop to time range of interest for display
        time_mask = (tfr_spike.times >= tcrop[0]) & (tfr_spike.times <= tcrop[1])
        times_cropped = tfr_spike.times[time_mask]
        data_cropped = data[:, :, time_mask]
        
        # Apply frequency range limits for display
        freq_mask = (freqs >= fmin) & (freqs <= fmax)
        freqs_display = freqs[freq_mask]
        data_display = data_cropped[:, freq_mask, :]
        
        # Set limits for better visualization
        if rescale_method in ['zscore', 'percent', 'db']:
            # For methods that produce values centered around zero
            vmin, vmax = np.percentile(data_display, [5, 95])
            # Make the colormap symmetric around zero
            abs_max = max(abs(vmin), abs(vmax))
            vmin, vmax = -abs_max, abs_max
        else:
            # For methods that don't produce values centered around zero
            vmin, vmax = np.percentile(data_display, [5, 95])
        
        # Plot the heatmap
        im = axes[ch_idx].pcolormesh(
            times_cropped, 
            freqs_display, 
            data_display[0, :, :],  # First dimension is epochs/average
            cmap=cmap,
            vmin=vmin,
            vmax=vmax
        )
        
        # Add colorbar
        cbar = fig.colorbar(im, ax=axes[ch_idx])
        cbar.set_label(colorbar_label)
        
        # Add labels and title
        axes[ch_idx].set_ylabel('Frequency (Hz)')
        if rescale_method == 'none':
            title = f'Channel: {ch_name}'
        else:
            title = f'Channel: {ch_name} ({rescale_method})'
        axes[ch_idx].set_title(title)
        
        # Add vertical line at t=0 (spike time)
        axes[ch_idx].axvline(x=0, color='black', linestyle='--', alpha=0.8)
        
        # # Add horizontal lines at classic frequency bands
        # band_colors = {'theta': 'blue', 'alpha': 'green', 'beta': 'orange', 'gamma': 'red'}
        # band_ranges = {'theta': (4, 8), 'alpha': (8, 13), 'beta': (13, 30), 'gamma': (30, 50)}
        
        # for band, (band_min, band_max) in band_ranges.items():
        #     if band_min >= fmin and band_min <= fmax:
        #         axes[ch_idx].axhline(y=band_min, color=band_colors[band], 
        #                             linestyle=':', alpha=0.6)
        #     if band_max >= fmin and band_max <= fmax:
        #         axes[ch_idx].axhline(y=band_max, color=band_colors[band], 
        #                             linestyle=':', alpha=0.6)
    
    # Add global x-axis label
    plt.xlabel('Time (s)')
    plt.tight_layout()
    
    # Add title with rescaling method
    if rescale_method != 'none':
        plt.suptitle(f'Time-Frequency Analysis ({rescale_method})', 
                   fontsize=12, y=1.02)
    else:
        plt.suptitle('Time-Frequency Analysis (unscaled)', 
                   fontsize=12, y=1.02)
    
    return fig
    

Plotting idea
- Plot individual ERP time series (option to select channels, time range)
- Add groupby variable for multiple files

- Night vs. day
    - split up the event
    - sar object has this information (timestamp)
    - use isday function in utils.py

Joseph
- rythmicity plot

In [ ]:
#epochs.plot(scalings = 'auto', n_epochs = 2)

# epochs_clean, rejection_info, rms_per_epoch = reject_epochs_by_rms(epochs, threshold_sd=1)


# plot_peri_spike_evoked(epochs)
# plot_peri_spike_evoked(epochs_clean)


In [ ]:
epochs = epoch_peri_spike(mne_obj)

plot_peri_spike_evoked(epochs)

# 1. Plain log10 transform
fig_log = plot_peri_spike_tfr(
    epochs, 
    l_freq=1, 
    h_freq=50, 
    tcrop=(-3, 3),
    rescale_method='log'
)

# 2. dB transform relative to baseline
fig_db = plot_peri_spike_tfr(
    epochs, 
    l_freq=1, 
    h_freq=50, 
    tcrop=(-3, 3),
    rescale_method='db'
)


In [28]:
# quick hack to look at peri-spike EEG

def analyze_peri_spike_tfr(mne_obj: mne.io.Raw, tmin=-10, tmax=5, tcrop=(-3, 3), baseline=(None, -5), l_freq=1, h_freq=50):
    """
    Analyze time-frequency representation around spikes from an MNE object.
    
    Parameters:
    -----------
    mne_obj : mne.io.Raw
        The MNE object containing the data and spike annotations
    tmin : float
        Start time relative to spike (in seconds)
    tmax : float
        End time relative to spike (in seconds)
    tcrop : tuple
        Time range to crop the data (in seconds)
    baseline : tuple
        Time range to compute baseline (in seconds)
    l_freq : float
        Lower frequency bound for filtering
    h_freq : float
        Upper frequency bound for filtering
    """
    raw = mne_obj.copy()
    
    # with joblib.parallel_backend('dask', scheduler_host=cluster_general.scheduler_address):
    # raw.filter(l_freq=l_freq, h_freq=h_freq)

    events, event_dict = mne.events_from_annotations(raw=raw)
    # logging.debug(f"Events: {events}")
    # logging.debug(f"Event dict: {event_dict}")

    # Process each spike type separately
    for channel_name in event_dict.keys():
        # logging.debug(f"Processing channel: {channel_name}")
        
        # Create spike-centered epochs
        epochs = mne.Epochs(
            raw,
            tmin=tmin,
            tmax=tmax,
            baseline=baseline,
            event_repeated='drop',
            # preload=True,
            event_id=channel_name,
        )
        epochs.drop_bad()
        epochs = random_subsample_epochs(epochs).load_data()
        epochs.crop(tmin=tcrop[0], tmax=tcrop[1])
        # with joblib.parallel_backend('dask', scheduler_host=cluster_general.scheduler_address):
        epochs.filter(l_freq=l_freq, h_freq=h_freq)

        epochs.compute_psd(fmin=1.0, fmax=80.0).plot(picks="eeg", average = True, exclude="bads", amplitude=False)
        # Plot the epochs
        epochs.plot_image([channel_name])


        
        # logging.info(f"Computing TFR: {channel_name}")
        # with joblib.parallel_config('dask', scheduler_host=cluster_general.scheduler_address):
        #     # epochs.tfr_stockwell(l_freq=l_freq, h_freq=h_freq)
        #     # tfr_spike = epochs.compute_tfr(freqs=(l_freq, h_freq), method='stockwell', average=True)
        #     # TODO take log?
        #     # TODO implement morlet over multitaper, it's cleaner. Follow documentation for exact
        
        tfr_spike = epochs.compute_tfr(freqs=np.arange(l_freq, h_freq, 1), n_cycles=np.arange(l_freq, h_freq, 1) * 2, method='morlet', use_fft=True, average=True)


        tfr_spike.data = np.log10(tfr_spike.data)

        vmin, vmax = np.percentile(tfr_spike.data, [5, 95])  # Using 5th and 95th percentiles

        tfr_spike.plot(picks=[channel_name], baseline=None, mode="logratio", title=channel_name, vlim=(vmin, vmax))


        # Return the objects from the first spike type
        return epochs#, tfr_spike


In [6]:
def plot_tfr_hilbert(epochs: mne.Epochs, freqs: list[float], bandwidth: float):
    data = np.empty((len(epochs), len(epochs.ch_names), freqs.size, epochs.times.size), dtype=complex)
    logging.info(f"data shape: {data.shape}")
    for idx, freq in enumerate(freqs):
        data[:, :, idx] = epochs.get_data()
        
    # TODO implement hilbert transform on narrowband filter

    # power = mne.time_frequency.EpochsTFRArray(epochs.info, data, epochs.times, freqs, method="hilbert")
    # power.average().plot(
    #     [0],
    #     baseline=(0.0, 0.1),
    #     mode="mean",
    #     vlim=(0, 0.1),
    #     axes=ax,
    #     show=False,

        

# fig, axs = plt.subplots(1, 3, figsize=(15, 5), sharey=True, layout="constrained")
# bandwidths = [1.0, 2.0, 4.0]
# for bandwidth, ax in zip(bandwidths, axs):
#     data = np.zeros(
#         (len(epochs), len(ch_names), freqs.size, epochs.times.size), dtype=complex
#     )
#     for idx, freq in enumerate(freqs):
#         # Filter raw data and re-epoch to avoid the filter being longer than
#         # the epoch data for low frequencies and short epochs, such as here.
#         raw_filter = raw.copy()
#         # NOTE: The bandwidths of the filters are changed from their defaults
#         # to exaggerate differences. With the default transition bandwidths,
#         # these are all very similar because the filters are almost the same.
#         # In practice, using the default is usually a wise choice.
#         raw_filter.filter(
#             l_freq=freq - bandwidth / 2,
#             h_freq=freq + bandwidth / 2,
#             # no negative values for large bandwidth and low freq
#             l_trans_bandwidth=min([4 * bandwidth, freq - bandwidth]),
#             h_trans_bandwidth=4 * bandwidth,
#         )
#         raw_filter.apply_hilbert()
#         epochs_hilb = Epochs(
#             raw_filter, events, tmin=0, tmax=n_times / sfreq, baseline=(0, 0.1)
#         )
#         data[:, :, idx] = epochs_hilb.get_data()
#     power = EpochsTFRArray(epochs.info, data, epochs.times, freqs, method="hilbert")
#     power.average().plot(
#         [0],
#         baseline=(0.0, 0.1),
#         mode="mean",
#         vlim=(0, 0.1),
#         axes=ax,
#         show=False,
#         colorbar=False,
#     )
#     n_cycles = "scaled by freqs" if not isinstance(n_cycles, int) else n_cycles
#     ax.set_title(
#         "Sim: Using narrow bandpass filter Hilbert,\n"
#         f"bandwidth = {bandwidth}, "
#         f"transition bandwidth = {4 * bandwidth}"
#     )

In [ ]:
# display(mne_obj)
epochs = analyze_peri_spike_tfr(mne_obj)

In [ ]:
epochs.plot(scalings = 'auto', n_epochs = 5)

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import logging
from typing import Literal, Optional, List, Dict, Any, Union, Tuple
from pathlib import Path

import mne
from mne.time_frequency import tfr_morlet

class MNEExperimentPlotter:
    """
    A class for creating various plots from a list of MNE objects with grouping capabilities.
    
    This class provides methods for creating different types of plots (evoked, TFR, etc.)
    from MNE data with consistent data processing, grouping, and contrasting.
    """

    def __init__(self, mne_objects: Union[mne.io.Raw, List[mne.io.Raw]]):
        """
        Initialize plotter with MNE Raw object(s).
        
        Parameters
        ----------
        mne_objects : mne.io.Raw or list[mne.io.Raw]
            Single MNE Raw object or list of MNE Raw objects
        """
        # Convert single MNE object to list
        if not isinstance(mne_objects, list):
            mne_objects = [mne_objects]
            
        self.mne_objects = mne_objects
        
        # Extract metadata from all objects
        self.metadata = []
        for obj in mne_objects:
            # Ensure temp dict exists
            if 'temp' not in obj.info:
                obj.info['temp'] = {}
            
            # Extract metadata from the object
            meta = {
                'animal_id': obj.info['temp'].get('animal_id', 'Unknown'),
                'file_path': obj.info['temp'].get('file_path', 'Unknown'),
                'sfreq': obj.info['sfreq'],
                'ch_names': obj.ch_names,
                'mne_obj': obj  # Store reference to the object itself
            }
            self.metadata.append(meta)
        
        # Collect all unique channel names
        self.all_channel_names = sorted(list(set([ch for obj in mne_objects for ch in obj.ch_names])))
        logging.info(f'all_channel_names: {self.all_channel_names}')

    def load_mne_objects(animal_ids, base_folder='./test-mnes'):
        """
        Load MNE objects from SpikeAnalysisResult files for specified animal IDs.
        
        Parameters
        ----------
        animal_ids : list of str
            List of animal IDs to load data for
        base_folder : str or Path
            Base folder containing the data files
            
        Returns
        -------
        list of mne.io.Raw
            List of loaded MNE Raw objects with metadata
        """
        import glob
        from pathlib import Path
        
        # Create a list to store all mne objects
        mne_objects = []
        
        # Get all files for each animal ID
        for animal_id in animal_ids:
            # Find all files matching the pattern for this animal
            file_pattern = f'{base_folder}/{animal_id}*'
            animal_files = glob.glob(file_pattern)
            
            print(f"Found {len(animal_files)} files for {animal_id}:")
            for file in animal_files:
                print(f"  - {file}")
                
                # Load each file and add to our list
                try:
                    reconstruct_sas = visualization.SpikeAnalysisResult.load_fif_and_json(file)
                    mne_obj = reconstruct_sas.result_mne
                    
                    # Store metadata in the 'temp' dictionary
                    if 'temp' not in mne_obj.info:
                        mne_obj.info['temp'] = {}
                    mne_obj.info['temp']['animal_id'] = animal_id
                    mne_obj.info['temp']['file_path'] = file
                    
                    # Add to our list
                    mne_objects.append(mne_obj)
                    print(f"  ✓ Successfully loaded")
                except Exception as e:
                    print(f"  ✗ Error loading: {str(e)}")
        
        print(f"\nTotal MNE objects loaded: {len(mne_objects)}")
        return mne_objects

    def get_objects_by_group(self, group_key: str) -> Dict[str, List[mne.io.Raw]]:
        """
        Group MNE objects by a metadata key
        
        Parameters
        ----------
        group_key : str
            Metadata key to group by (e.g. 'animal_id')
            
        Returns
        -------
        Dict[str, List[mne.io.Raw]]
            Dictionary mapping group values to lists of MNE objects
        """
        groups = {}
        for meta in self.metadata:
            value = meta.get(group_key, 'Unknown')
            if value not in groups:
                groups[value] = []
            groups[value].append(meta['mne_obj'])
        return groups

    def create_epochs(self, tmin: float = -0.2, tmax: float = 0.5, 
                      baseline: Tuple[Optional[float], Optional[float]] = (None, 0),
                      event_id: Optional[Dict[str, int]] = None) -> Dict[str, List[mne.Epochs]]:
        """
        Create epochs for all MNE objects
        
        Parameters
        ----------
        tmin : float
            Start time before event
        tmax : float
            End time after event
        baseline : tuple
            Baseline correction period
        event_id : dict
            Event IDs to include
            
        Returns
        -------
        Dict[str, List[mne.Epochs]]
            Dictionary mapping animal_ids to lists of Epochs objects
        """
        grouped_epochs = {}
        
        for meta in self.metadata:
            animal_id = meta['animal_id']
            raw = meta['mne_obj']
            
            try:
                # Extract events from annotations
                events, event_dict = mne.events_from_annotations(raw)
                
                # Use provided event_id if given, otherwise use all events
                use_event_id = event_id if event_id else event_dict
                
                # Create epochs
                epochs = mne.Epochs(
                    raw, 
                    events, 
                    event_id=use_event_id,
                    tmin=tmin, 
                    tmax=tmax, 
                    baseline=baseline,
                    preload=True
                )
                
                if animal_id not in grouped_epochs:
                    grouped_epochs[animal_id] = []
                
                grouped_epochs[animal_id].append(epochs)
                
            except Exception as e:
                logging.warning(f"Error creating epochs for {animal_id}: {str(e)}")
        
        return grouped_epochs

    def plot_evoked_by_group(self, group_key: str = 'animal_id', 
                        tmin: float = -0.2, tmax: float = 0.5,
                        baseline: Tuple[Optional[float], Optional[float]] = (None, 0),
                        event_id: Optional[Dict[str, int]] = None,
                        picks: Optional[List[str]] = None,
                        combine: str = 'mean',
                        title: Optional[str] = None,
                        figsize: Optional[Tuple[float, float]] = None) -> Tuple[plt.Figure, List[plt.Axes]]:
        """
        Plot evoked responses grouped by a metadata key
        
        Parameters
        ----------
        group_key : str
            Metadata key to group by (e.g. 'animal_id')
        tmin : float
            Start time before event
        tmax : float
            End time after event
        baseline : tuple
            Baseline correction period
        event_id : dict
            Event IDs to include
        picks : list
            Channels to include
        combine : str
            How to combine channels ('mean', 'median', etc.)
        title : str
            Figure title
        figsize : tuple
            Figure size
            
        Returns
        -------
        Tuple[plt.Figure, List[plt.Axes]]
            Figure and axes objects
        """
        # Get objects grouped by the specified key
        grouped_objects = self.get_objects_by_group(group_key)
        
        # Create figure
        if figsize is None:
            figsize = (10, 8 * len(grouped_objects))
        fig, axes = plt.subplots(len(grouped_objects), 1, figsize=figsize, sharex=True)
        if len(grouped_objects) == 1:
            axes = [axes]
        
        # Process each group
        for ax_idx, (group_value, objects) in enumerate(grouped_objects.items()):
            ax = axes[ax_idx]
            
            # List to store evoked objects
            evoked_list = []
            
            # Process each object in the group
            for raw in objects:
                try:
                    # Extract events
                    events, event_dict = mne.events_from_annotations(raw)
                    
                    # Use provided event_id if given, otherwise use all events
                    use_event_id = event_id if event_id else event_dict
                    
                    # Create epochs with channel selection
                    epochs = mne.Epochs(
                        raw, 
                        events, 
                        event_id=use_event_id,
                        tmin=tmin, 
                        tmax=tmax, 
                        baseline=baseline,
                        preload=True,
                        picks=picks  # Apply picks here when creating epochs
                    )
                    
                    # Get evoked response
                    if len(epochs) > 0:
                        evoked = epochs.average()
                        evoked_list.append(evoked)
                    
                except Exception as e:
                    logging.warning(f"Error processing {group_value}: {str(e)}")
            
            # Plot evoked responses
            if evoked_list:
                # Average across all evoked objects in this group
                all_evoked = mne.combine_evoked(evoked_list, weights='equal')
                
                # Plot the data as regular time series
                all_evoked.plot(axes=ax, show=False, titles=None, 
                            time_unit='s', gfp=True, 
                            spatial_colors=True)
                
                # Add group title
                ax.set_title(f"{group_key}: {group_value}")
            else:
                ax.text(0.5, 0.5, f"No valid data for {group_value}", 
                    ha='center', va='center', transform=ax.transAxes)
        
        # Set overall title
        if title:
            fig.suptitle(title, fontsize=16)
            
        plt.tight_layout()
        return fig, axes

    def compute_tfr(self, epochs: mne.Epochs, freqs: np.ndarray, 
                   n_cycles: Union[float, List[float], np.ndarray] = None,
                   picks: Optional[List[str]] = None,
                   return_itc: bool = False) -> mne.time_frequency.AverageTFR:
        """
        Compute TFR for epochs using Morlet wavelets
        
        Parameters
        ----------
        epochs : mne.Epochs
            Epochs object
        freqs : array
            Frequencies of interest
        n_cycles : float or array
            Number of cycles in Morlet wavelet
        picks : list
            Channels to include
        return_itc : bool
            Whether to return inter-trial coherence
            
        Returns
        -------
        mne.time_frequency.AverageTFR
            TFR object
        """
        # Set default n_cycles if not provided
        if n_cycles is None:
            n_cycles = freqs / 2
        
        # Compute TFR
        power, itc = tfr_morlet(epochs, freqs=freqs, n_cycles=n_cycles, 
                               use_fft=True, return_itc=True, picks=picks)
        
        return itc if return_itc else power

    def plot_tfr_contrast(self, group_key: str = 'animal_id',
                        tmin: float = -0.5, tmax: float = 1.0,
                        baseline: Tuple[Optional[float], Optional[float]] = (None, 0),
                        event_id: Optional[Dict[str, int]] = None,
                        freqs: np.ndarray = np.linspace(4, 40, 20),
                        n_cycles: Optional[Union[float, List[float], np.ndarray]] = None,
                        picks: Optional[List[str]] = None,
                        mode: str = 'zscore',
                        title: Optional[str] = None,
                        figsize: Optional[Tuple[float, float]] = None) -> Tuple[plt.Figure, List[plt.Axes]]:
        """
        Plot TFR contrast between groups
        
        Parameters
        ----------
        group_key : str
            Metadata key to group by (e.g. 'animal_id')
        tmin : float
            Start time before event
        tmax : float
            End time after event
        baseline : tuple
            Baseline correction period
        event_id : dict
            Event IDs to include
        freqs : array
            Frequencies of interest
        n_cycles : float or array
            Number of cycles in Morlet wavelet
        picks : list
            Channels to include
        mode : str
            Mode for baseline correction ('zscore', 'mean', 'ratio', 'logratio', 'percent')
        title : str
            Figure title
        figsize : tuple
            Figure size
            
        Returns
        -------
        Tuple[plt.Figure, List[plt.Axes]]
            Figure and axes objects
        """
        # Get objects grouped by the specified key
        grouped_objects = self.get_objects_by_group(group_key)
        groups = list(grouped_objects.keys())
        
        # We need at least 2 groups for contrast
        if len(groups) < 2:
            raise ValueError(f"Need at least 2 groups for contrast, but only found: {groups}")
        
        # Get channel list to use - intersection of channels in all objects
        all_channels = [obj.ch_names for group in grouped_objects.values() for obj in group]
        common_channels = set(all_channels[0])
        for ch_list in all_channels[1:]:
            common_channels &= set(ch_list)
        common_channels = sorted(list(common_channels))
        
        if picks:
            common_channels = [ch for ch in common_channels if ch in picks]
        
        if not common_channels:
            raise ValueError("No common channels found across all objects")
        
        logging.info(f"Using common channels: {common_channels}")
        
        # Dictionary to store TFR results for each group
        tfr_by_group = {}
        
        # Process each group
        for group, objects in grouped_objects.items():
            # List to store TFR objects
            tfr_list = []
            
            # Process each object in the group
            for raw in objects:
                try:
                    # Extract events
                    events, event_dict = mne.events_from_annotations(raw)
                    
                    # Use provided event_id if given, otherwise use all events
                    use_event_id = event_id if event_id else event_dict
                    
                    # Create epochs
                    epochs = mne.Epochs(
                        raw, 
                        events, 
                        event_id=use_event_id,
                        tmin=tmin, 
                        tmax=tmax, 
                        baseline=baseline,
                        preload=True,
                        picks=common_channels
                    )
                    
                    # Compute TFR
                    if len(epochs) > 0:
                        tfr = self.compute_tfr(epochs, freqs, n_cycles, picks=common_channels)
                        tfr_list.append(tfr)
                    
                except Exception as e:
                    logging.warning(f"Error processing {group}: {str(e)}")
            
            # Average TFRs for this group
            if tfr_list:
                # Create a grand average TFR
                tfr_avg = tfr_list[0].copy()
                if len(tfr_list) > 1:
                    for tfr in tfr_list[1:]:
                        tfr_avg._data += tfr._data
                    tfr_avg._data /= len(tfr_list)
                
                # Apply baseline correction
                tfr_avg.apply_baseline(baseline=baseline, mode=mode)
                
                # Store result
                tfr_by_group[group] = tfr_avg
        
        # Create pairwise contrasts
        contrasts = []
        group_pairs = []
        for i, group1 in enumerate(groups):
            for group2 in groups[i+1:]:
                if group1 in tfr_by_group and group2 in tfr_by_group:
                    # Create a contrast TFR
                    contrast = tfr_by_group[group1].copy()
                    contrast._data -= tfr_by_group[group2]._data
                    
                    contrasts.append(contrast)
                    group_pairs.append((group1, group2))
        
        # Determine subplot layout
        n_channels = len(common_channels)
        n_contrasts = len(contrasts)
        
        # Create figure
        if figsize is None:
            figsize = (4 * n_contrasts, 3 * n_channels)
        
        fig, axes = plt.subplots(n_channels, n_contrasts, figsize=figsize)
        
        # Handle case of single channel or contrast
        if n_channels == 1 and n_contrasts == 1:
            axes = np.array([[axes]])
        elif n_channels == 1:
            axes = axes.reshape(1, -1)
        elif n_contrasts == 1:
            axes = axes.reshape(-1, 1)
        
        # Plot each contrast for each channel
        for contrast_idx, (contrast, group_pair) in enumerate(zip(contrasts, group_pairs)):
            for ch_idx, ch_name in enumerate(common_channels):
                ax = axes[ch_idx, contrast_idx]
                
                # Extract data for this channel
                ch_idx_in_tfr = contrast.ch_names.index(ch_name)
                data = contrast.data[ch_idx_in_tfr]
                
                # Plot TFR
                im = ax.pcolormesh(contrast.times, contrast.freqs, data,
                                  cmap='RdBu_r', vmin=-np.max(np.abs(data)), vmax=np.max(np.abs(data)))
                
                # Add colorbar for rightmost column
                if contrast_idx == n_contrasts - 1:
                    plt.colorbar(im, ax=ax)
                
                # Set labels
                if ch_idx == 0:  # Top row
                    ax.set_title(f"{group_pair[0]} - {group_pair[1]}")
                
                if contrast_idx == 0:  # Leftmost column
                    ax.set_ylabel(f"{ch_name}\nFreq (Hz)")
                
                if ch_idx == n_channels - 1:  # Bottom row
                    ax.set_xlabel("Time (s)")
        
        # Set overall title
        if title:
            fig.suptitle(title, fontsize=16)
            
        plt.tight_layout()
        return fig, axes

In [6]:
# Load MNE objects using the static method
animal_ids = ['A5', 'A10']
mne_objects = MNEExperimentPlotter.load_mne_objects(animal_ids, base_folder='./test-mnes')

# After loading your MNE objects as shown in your existing code
plotter = MNEExperimentPlotter(mne_objects)

# To plot evoked responses grouped by animal ID
fig, axes = plotter.plot_evoked_by_group(group_key='animal_id')

# To plot TFR contrast between animals
fig, axes = plotter.plot_tfr_contrast(
    group_key='animal_id',
    tmin=-0.5, 
    tmax=1.0,
    freqs=np.linspace(4, 40, 20),
    mode='zscore'
)

NameError: name 'mne_objects' is not defined